In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Run all the operations in one cell.
#Importing necessary libraries
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import pandas as pd



#Data separation
train_dir = '/kaggle/input/horses-or-humans-dataset/horse-or-human/horse-or-human/train'
val_dir = '/kaggle/input/horses-or-humans-dataset/horse-or-human/horse-or-human/validation'
#Input dimensions
width_size = 300
height_size = 300
color = 3
epoch_number = 5
#For train_directory
#To use the ImageDataGenerator, you simply ensure that your directory structure has
#a set of named subdirectories, with each subdirectory being a label.
#Applying image argumentation by applying other type of transformation
train_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(width_size, height_size),class_mode='categorical')

#For validation_directory
#To use the ImageDataGenerator, you simply ensure that your directory structure has
#a set of named subdirectories, with each subdirectory being a label.
#Applying image argumentation by applying other type of transformation
val_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
val_generator = val_datagen.flow_from_directory(val_dir,target_size=(width_size, height_size),class_mode='categorical')
class_labels = val_generator.class_indices

#CNN Architecture
#Initialize the model
model = Sequential()

#Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width_size, height_size, color)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

#Flatten the output and add dense layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))


#Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Print the model summary
model.summary()

#Train the model using the training data
model.fit(train_generator, epochs=epoch_number, validation_data= val_generator)

#Saving the model
#Save the model to a file
model.save('/kaggle/working/human_or_Horse_model.h5')            
# Load the saved model
model = load_model('/kaggle/working/human_or_Horse_model.h5')




In [ ]:
# Load the saved model and make predictions
loaded_model = load_model('/kaggle/working/human_or_Horse_model.h5')

#Since a model was trained using RGB color,
#An external loaded image must be in .jpg format
#Image converter
def convert_png_to_jpg(png_path, jpg_path):
    image = Image.open(png_path)
    rgb_image = image.convert("RGB")
    rgb_image.save(jpg_path, "JPEG")
    
#Loading an external data
image_png = "/kaggle/input/horses-or-humans-dataset/horse-or-human/train/humans/human07-10.png"
image_jpg = "/kaggle/working/testing.jpg"
#function calling
convert_png_to_jpg(image_png, image_jpg)

#Load a .jpg image now
image = Image.open(image_jpg)

#preprocess the input
#Input dimensions
width_size = 300
height_size = 300
image = image.resize((width_size, height_size))
image = np.array(image)
image = np.expand_dims(image, axis = 0)
image_tensor = np.vstack([image])
predictions = loaded_model.predict(image_tensor)
#print a maxmum value
predicted_labels = np.argmax(predictions, axis=1)
#print the disease predicted
for i in predicted_labels:
# Iterate over the dictionary items and find the key for the desired value
    for key, value in class_labels.items():
        if value == i:
            print("{} : {}".format(i,key))